# Loading Multi-photon Calcium Imaging Data

Cellular Calcium activity recorded using a multi-photon imaging.

## Relevant ALF objects
* mpci
* mpciROIs
* mpciROITypes
* mpciMeanImage
* mpciStack

## Finding sessions with imaging data

In [ ]:
# Find mesoscope imaging sessions in V1
from one.api import ONE
one = ONE()

# atlas_id, atlas_name
res = one.alyx.rest('sessions', 'list', procedures='Imaging', django='field_of_view__imaging_type__name,mesoscope')
eids = one.search(procedures='Imaging', django='field_of_view__imaging_type__name,mesoscope', atlas_acronym='AIp6b')
res = one.alyx.rest('fields-of-view', 'list', imaging_type='mesoscope', atlas_acronym='AIp6b')
fov00  = one.load_collection(res[0]['session'], '*' + res[0]['name'])
eid = 'a5550a8e-2484-4539-b7f0-8e5f829d0ba7'
fovs = one.alyx.rest('fields-of-view', 'list', imaging_type='mesoscope', atlas_id=187, session=eid)

# Search by layer
# Only sessions with GCaMP mice
# Only sessions with single plane / N planes
JSON field = slice_id
field = part_of_stack


# List the number of fields of view (FOVs) recorded during a session

In [ ]:
from one.api import ONE
one = ONE()
eid = 'b1ca324f-5db7-4106-8be2-0dd9cce17648'

fov_folders = one.list_collections(eid, collection='alf/FOV_*')
fovs = sorted(map(lambda x: int(x[-2:]), fov_folders))
nFOV = len(fovs)

## Loading ROI activity for a single session

In [ ]:
# Loading ROI activity for a single FOV
ROI_data_00 = one.load_collection(eid, 'alf/FOV_00', object=['mpci', 'mpciROI', 'mpciROITypes'])
print(ROI_data_00.keys())

# Loading ROI activity for all FOVs
all_ROI_data = one.load_collection(eid, 'alf/FOV_*', object=['mpci', 'mpciROI', 'mpciROITypes'])
print(all_ROI_data.keys())
print(all_ROI_data.FOV_00.keys())

## More details
* [Description of mesoscope datasets](https://docs.google.com/document/d/1OqIqqakPakHXRAwceYLwFY9gOrm8_P62XIfCTnHwstg/edit#heading=h.nvzaz0fozs8h)
* [Loading raw mesoscope data](./loading_raw_mesoscope_data.ipynb)